In [52]:
%run model_functions.ipynb

In [155]:
import stanfordnlp

In [61]:
sentences = pd.read_csv('data/raw/sentences.csv', encoding='ISO-8859-1')
display(len(sentences))
sentences.head()

16183

,sentence_id,paragraph_id,sentence
0,0,-1,"Latest corporate unbundler reveals laid-back approach: Roland Franklin, who is leading a 697m pound break-up bid for DRG, talks to Frank Kane"
1,1,-1,By FRANK KANE
2,2,3,"IT SEEMS that Roland Franklin, the latest unbundler to appear in the UK, has made a fatal error in the preparation of his £697m break-up bid for stationery and packaging group DRG."
3,3,3,He has not properly investigated the target's dining facilities.
4,4,4,"The 63-year-old head of Pembridge Investments, through which the bid is being mounted says, rule number one in this business is: the more luxurious the luncheon rooms at headquarters, the more inefficient the business."


..\aten\src\ATen\native\LegacyDefinitions.cpp:14: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


2

In [ ]:
def parse_dependencies(sentence, sentence_id, nlp_pipeline):
    doc = nlp_pipeline(sentence)
    
    dependencies = []
    for sentence in doc.sentences:
        for dep in sentence.dependencies:
            governor, g_i = dep[0].lemma, int(dep[0].index) - 1
            if governor is None:
                continue
            dependent, d_i = dep[2].lemma, int(dep[2].index) - 1
            dependencies.append((sentence_id, governor, g_i, dependent, d_i))
            
    return dependencies

def parse_all_deps(sentences, use_saved=True, download_models=False):
    if download_models:
        stanfordnlp.download('en')   # This downloads the English models for the neural pipeline

    nlp_pipeline = stanfordnlp.Pipeline() # This sets up a default neural pipeline in English

    with open('data/all_dependencies.csv', 'w', encoding='utf-8') as f:
        csv_writer = csv.writer(f)
        for i, sentence in enumerate(sentences):
            print('Finding dependencies for sentence {}'.format(i))
            for dep in parse_dependencies(sentence, i, nlp_pipeline):
                csv_writer.writerow(dep)

parse_all_deps(list(sentences['sentence']), nlp)

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': 'C:\\Users\\ezutp\\stanfordnlp_resources\\en_ewt_models\\en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': 'C:\\Users\\ezutp\\stanfordnlp_resources\\en_ewt_models\\en_ewt_tagger.pt', 'pretrain_path': 'C:\\Users\\ezutp\\stanfordnlp_resources\\en_ewt_models\\en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': 'C:\\Users\\ezutp\\stanfordnlp_resources\\en_ewt_models\\en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': 'C:\\Users\\ezutp\\stanfordnlp_resources\\en_ewt_models\\en_ewt_parser.pt', 'pretrain_path': 'C:\\Users\\ezutp\

..\aten\src\ATen\native\LegacyDefinitions.cpp:14: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


Finding dpendencies for sentence 1


..\aten\src\ATen\native\LegacyDefinitions.cpp:14: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


Finding dpendencies for sentence 2


...

Finding dpendencies for sentence 1417


..\aten\src\ATen\native\LegacyDefinitions.cpp:14: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


Finding dpendencies for sentence 1418


..\aten\src\ATen\native\LegacyDefinitions.cpp:14: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


Finding dpendencies for sentence 1419


In [167]:
pd.read_csv('data/all_dependencies.csv',
             names=['governor', 'g_id', 'dependent', 'd_id'])

,governor,g_id,dependent,d_id
0,unbundler,2,latest,0
0,unbundler,2,corporate,1
0,reveal,3,unbundler,2
0,approach,7,lay,4
0,back,6,-,5
0,approach,7,back,6
0,reveal,3,approach,7
0,reveal,3,:,8
0,Roland,0,Franklin,1
0,Roland,0,",",2


In [53]:
dependencies = pd.read_csv('data/dependencies.csv', index_col='sentence_id')
display(len(dependencies))
dependencies.head()

151124

,governor,dependent,relation
sentence_id,,,
0,Roland,made,nsubj
0,Franklin,Roland,flat
0,latest,unbundler,amod
0,unbundler,Roland,appos
0,appear,unbundler,acl


In [55]:
pd.read_csv('data/raw/sentences.csv', encoding='iso-8859-1').iloc[6582]

sentence_id     6582                                                                                                                                                                                                                                                                 
paragraph_id    2004                                                                                                                                                                                                                                                                 
sentence        Today, Sartre's voluntarism is to some extent returning to favour as the result of a desire to retrieve the categories of agency and the subject, which goes together with the wish to get out of the apparently totalizing systems of Adorno, Althusser or Foucault.
Name: 6582, dtype: object

In [43]:
dependencies.nunique()

governor     16367
dependent    11442
relation        39
dtype: int64

In [47]:
dependencies[dependencies['governor']=='Roland']

,governor,dependent,relation
sentence_id,,,
0,Roland,made,nsubj
6582,Roland,way,nmod:poss
8551,Roland,made,nsubj
15133,Roland,way,nmod:poss


In [45]:
dependencies[dependencies.duplicated(keep=False)].sort_values('sentence_id')

,governor,dependent,relation
sentence_id,,,
0,Roland,made,nsubj
0,DRG,bid,nmod
0,group,DRG,compound
0,packaging,stationery,conj
0,bid,preparation,nmod
0,up,bid,compound
0,break,up,compound
0,preparation,made,obl
0,stationery,DRG,compound


In [156]:
words = pd.DataFrame([], columns=['word', 'dep_word', 'sentence_id']).set_index('sentence_id')
words

,word,dep_word
sentence_id,,


In [157]:
words.append(
    dependencies[['governor', 'dependent']]
    .rename(columns={
            'governor': 'word',
            'dependent': 'dep_word'
    })).append(
    dependencies[['dependent', 'governor']]
    .rename(columns={
            'governor': 'word',
            'dependent': 'dep_word'
    })).drop_duplicates()

C:\Users\ezutp\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,dep_word,word
sentence_id,,
0,made,Roland
0,Roland,Franklin
0,unbundler,latest
0,Roland,unbundler
0,unbundler,appear
0,appear,UK
0,SEEMS,made
0,error,fatal
0,made,error
